In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import cv2
import random
from random import randint
import time

import warnings
warnings.simplefilter(action='ignore', category=Warning)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<center style="font-family:verdana;"><h1 style="font-size:200%; padding: 10px; background: #A0522D;"><b style="color:white;">The UW-Madison Carbone Cancer Center</b></h1></center>


"The UW-Madison Carbone Cancer Center is a pioneer in MR-Linac based radiotherapy, and has treated patients with MRI guided radiotherapy based on their daily anatomy since 2015. UW-Madison has generously agreed to support this project which provides anonymized MRIs of patients treated at the UW-Madison Carbone Cancer Center. The University of Wisconsin-Madison is a public land-grant research university in Madison, Wisconsin. The Wisconsin Idea is the university's pledge to the state, the nation, and the world that their endeavors will benefit all citizens."

https://www.kaggle.com/competitions/uw-madison-gi-tract-image-segmentation/overview/description

![](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcREyFel3_YgmecDDOs0Zryoo_AqaNyT6LXi9A&usqp=CAU)seekpng.com

<h1><span class="label label-default" style="background-color:#A0522D;border-radius:100px 100px; font-weight: bold; font-family:Garamond; font-size:20px; color:white; padding:10px">EfficientNet B0</span></h1><br>


We have Efficienet B0 (zero) to B7. I'm just trying to understand them all.

#Yes, I wish it. Spoiler alert: I wasn't able to explain it.

![](https://i.ytimg.com/vi/3svIm5UC94I/maxresdefault.jpg)youtube.com

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import os
import pathlib
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing import image_dataset_from_directory
import itertools

In [ ]:
#Code by Gonzalo Recio  https://www.kaggle.com/code/gonzalorecioc/alzheimer-brain-mri-classifier-effnetb0-99-acc

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
for dirpath, dirnames, filenames in os.walk("/kaggle/input"):
    print(f"{len(dirnames)} dirs and {len(filenames)} images in '{dirpath}'")

In [ ]:
#Code by Gonzalo Recio  https://www.kaggle.com/code/gonzalorecioc/alzheimer-brain-mri-classifier-effnetb0-99-acc

data_dir = "/kaggle/input/uw-madison-gi-tract-image-segmentation/"
train_dir = pathlib.Path("/kaggle/input/uw-madison-gi-tract-image-segmentation/train/") 
class_names = np.array(sorted([item.name for item in train_dir.glob('*')]))
print(class_names)

In [ ]:
import torch
from torch.utils.data import Dataset, random_split, DataLoader
import torch.nn.functional as F
import torch.nn as nn

from PIL import Image
from scipy import ndimage

import torchvision
import torchvision.models as models
import torchvision.transforms as T
from torchvision.utils import make_grid
from torchvision.datasets.utils import download_url
from torchvision.datasets import ImageFolder

from tqdm.notebook import tqdm

from sklearn.metrics import f1_score

In [ ]:
#Code by Kamal Das https://www.kaggle.com/kmldas/beginner-s-guide-image-augmentation-transforms

DATA_DIR = '../input/uw-madison-gi-tract-image-segmentation'


TRAIN_DIR = DATA_DIR + '/train'                           
TEST_DIR = DATA_DIR + '/test'                             

TRAIN_CSV = DATA_DIR + '/train.csv'                     
SUB_CSV = DATA_DIR + '/sample_submission.csv'
#TRAIN_DIR = DATA_DIR + '/images/content/images'

In [ ]:
data_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/train.csv', encoding='utf8')
data_df.head()

In [ ]:
#Code by Kamal Das https://www.kaggle.com/kmldas/beginner-s-guide-image-augmentation-transforms

labels_names=data_df["id"].unique()
labels_sorted=labels_names.sort()

labels = dict(zip(range(len(labels_names)),labels_names))
labels 

#To use numbers instead of names for labels. Lets add the numbers as labels to the dataframe

In [ ]:
#Code by Kamal Das https://www.kaggle.com/kmldas/beginner-s-guide-image-augmentation-transforms

lbl=[]
for i in range(len(data_df["id"])):
    temp=list(labels.values()).index(data_df.id[i])
    lbl.append(temp)

    
data_df['lbl'] = lbl
#data_df['lbl'] = data_df['lbl'].astype(str)
data_df.head()

In [ ]:
#Code by Kamal Das https://www.kaggle.com/kmldas/beginner-s-guide-image-augmentation-transforms

path_img=[]
for i in range(len(data_df["id"])):
    temp=TRAIN_DIR + "/" + str(data_df.id[i])  + ".png"
    path_img.append(temp)

data_df['path_img'] =path_img
data_df.head()

In [ ]:
num_images = len(data_df["id"])
print('Number of images in Training file:', num_images)
no_labels=len(labels_names)
print('Number of Organ Cells images in Training file:', no_labels)

#Mean Dice Coefficient and 3D Hausdorff

This competition is evaluated on the mean Dice coefficient and 3D Hausdorff distance. The Dice coefficient can be used to compare the pixel-wise agreement between a predicted segmentation and its corresponding ground truth. 

In [ ]:
bar = data_df["class"].value_counts(ascending=True).plot.barh(figsize = (10,8))
plt.title("Distribution of the Organ Cells Images", fontsize = 20)
bar.tick_params(labelsize=16)
plt.show()

#Plan B since the initial Plan with torch didn't work as I expected.

#Model Tuning

Split and preprocess data

In [ ]:
#Code by Gonzalo Recio https://www.kaggle.com/code/gonzalorecioc/alzheimer-brain-mri-classifier-effnetb0-99-acc

# For replicable results
SEED = 0
# Size of the images is (128,128)
IMAGE_SIZE = (128, 128)
# Default batch size
BATCH_SIZE = 32
# Images are grayscale
COLOR_MODE = "grayscale"
# 20% test split
VAL_SPLIT = 0.2

tf.random.set_seed(SEED)
np.random.seed(SEED)
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    label_mode='categorical',
    validation_split=VAL_SPLIT,
    subset="training",
    seed=SEED,
    color_mode=COLOR_MODE,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)
valid_data = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=VAL_SPLIT,
    subset="validation",
    label_mode='categorical',
    seed=SEED,
    color_mode=COLOR_MODE,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
)

#Model creation and compilation (EfficientNet B0)

In [ ]:
#Code by Gonzalo Recio https://www.kaggle.com/code/gonzalorecioc/alzheimer-brain-mri-classifier-effnetb0-99-acc

base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = True
inputs = tf.keras.layers.Input(shape=(IMAGE_SIZE+(1,)), name="input_layer")
# Efficient net model has the normalizing layer builtin
x = base_model(inputs)
x = tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
outputs = tf.keras.layers.Dense(len(class_names), activation="softmax", name="output_layer")(x)

In [ ]:
model = tf.keras.Model(inputs, outputs)

In [ ]:
#Code by Gonzalo Recio https://www.kaggle.com/code/gonzalorecioc/alzheimer-brain-mri-classifier-effnetb0-99-acc

# Default Learning rate
LR = 0.001

model.compile(loss="categorical_crossentropy", 
                optimizer=tf.keras.optimizers.Adam(learning_rate=LR), 
                metrics=["accuracy"])

In [ ]:
model.summary()

In [ ]:
#Code by Gonzalo Recio https://www.kaggle.com/code/gonzalorecioc/alzheimer-brain-mri-classifier-effnetb0-99-acc

# Epochs
EPOCHS = 50
history = model.fit(train_data,
                      validation_data=valid_data,
                      epochs=EPOCHS,
                      verbose=False
                      )

#Definitely, Epochs don't like me.  And due to my Epochs flaw my version has failed/or been cancelled.

In [ ]:
#Code by Gonzalo Recio https://www.kaggle.com/code/gonzalorecioc/alzheimer-brain-mri-classifier-effnetb0-99-acc

pd.DataFrame(history.history).plot(figsize=(10, 7));

In [ ]:
#Code by Gonzalo Recio https://www.kaggle.com/code/gonzalorecioc/alzheimer-brain-mri-classifier-effnetb0-99-acc

model.evaluate(valid_data)

#Final Thoughts

I don't have any at all

![](https://i.etsystatic.com/22180462/r/il/1089ca/2910732405/il_794xN.2910732405_lrok.jpg)etsy.com

#Acknowledgements: 

Gonzalo Recio https://www.kaggle.com/code/gonzalorecioc/alzheimer-brain-mri-classifier-effnetb0-99-acc

Kamal Das https://www.kaggle.com/kmldas/beginner-s-guide-image-augmentation-transforms